In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [14]:
file_path = "/Users/akshitsanoria/Desktop/retail/data/main_seg.xlsx"
df = pd.read_excel(file_path, engine="calamine")


In [15]:
df.head()

,Unnamed: 0,order_id,order_datetime,customer_id,country,order_revenue,item_count,unique_products,order_number
0,1006,491725,2009-12-14 08:34:00,12346,United Kingdom,45.0,10,1,1
1,1017,491742,2009-12-14 11:00:00,12346,United Kingdom,22.5,5,1,2
2,1019,491744,2009-12-14 11:02:00,12346,United Kingdom,22.5,5,1,3
3,1361,492718,2009-12-18 10:47:00,12346,United Kingdom,22.5,5,1,4
4,1363,492722,2009-12-18 10:55:00,12346,United Kingdom,1.0,1,1,5


In [16]:
total_revenue = df["order_revenue"].sum()
print(f"Total Revenue: {total_revenue}")

Total Revenue: 8832003.274


In [17]:
aov = (
    df["order_revenue"].sum() /
    df["order_id"].nunique()
)
print(f"Average Order Value (AOV): {aov}")

Average Order Value (AOV): 459.64107593026284


In [18]:
orders_per_customer = (
    df["order_id"].nunique() /
    df["customer_id"].nunique()
)
print(f"Orders per Customer: {orders_per_customer}")

Orders per Customer: 4.454102920723226


In [19]:
revenue_per_customer = (
    df["order_revenue"].sum() /
    df["customer_id"].nunique()
)
print(f"Revenue per Customer: {revenue_per_customer}")

Revenue per Customer: 2047.28865878535


In [20]:
df["customer_type"] = np.where(
    df["order_number"] == 1,
    "New",
    "Returning"
)

revenue_by_type = (
    df
    .groupby("customer_type")["order_revenue"]
    .sum()
)
print(f"Revenue by type: {revenue_by_type}")

Revenue by type: customer_type
New          1806162.117
Returning    7025841.157
Name: order_revenue, dtype: float64


In [21]:
df["order_month"] = (
    df["order_datetime"].dt.to_period("M")
)

monthly_revenue = (
    df.groupby("order_month")["order_revenue"]
               .sum()
               .reset_index()
)
print(f"Monthly Revenue: {monthly_revenue}")

Monthly Revenue:    order_month  order_revenue
0      2009-12     686654.160
1      2010-01     557319.062
2      2010-02     506371.066
3      2010-03     699608.991
4      2010-04     594609.192
5      2010-05     599985.790
6      2010-06     639066.580
7      2010-07     591636.740
8      2010-08     604242.650
9      2010-09     831615.001
10     2010-10    1036680.000
11     2010-11    1172336.042
12     2010-12     311878.000


In [22]:
repeat_rate = (
    (df["order_number"] > 1).sum() /
    df["customer_id"].nunique()
)
print(f"Repeat rate: {repeat_rate}")

Repeat rate: 3.4541029207232268


In [23]:
customer_revenue = (
    df.groupby("customer_id")["order_revenue"]
               .sum()
               .sort_values(ascending=False)
)

top_10pct_revenue_share = (
    customer_revenue.head(int(0.1 * len(customer_revenue))).sum() /
    customer_revenue.sum()
)
print(f"Top 10 Revenue share: {top_10pct_revenue_share}")

Top 10 Revenue share: 0.5983013006297035


In [24]:
df["order_month"] = (
    df["order_datetime"].dt.to_period("M")
)

monthly_active_customers = (
    df
    .groupby("order_month")["customer_id"]
    .nunique()
    .reset_index(name="active_customers")
)
customer_orders = (
    df.groupby("customer_id")["order_id"]
               .nunique()
)

repeat_purchase_rate = (
    (customer_orders > 1).sum() / customer_orders.count()
)

df["cohort_month"] = (
    df.groupby("customer_id")["order_month"]
               .transform("min")
)
cohort_data = (
    df
    .groupby(["cohort_month", "order_month"])["customer_id"]
    .nunique()
    .reset_index()
)
cohort_data["cohort_index"] = (
    cohort_data["order_month"] - cohort_data["cohort_month"]
).apply(lambda x: x.n)

cohort_pivot = cohort_data.pivot_table(
    index="cohort_month",
    columns="cohort_index",
    values="customer_id"
)

cohort_retention = cohort_pivot.divide(cohort_pivot.iloc[:, 0], axis=0)


In [25]:
last_purchase = (
    df.groupby("customer_id")["order_datetime"]
               .max()
)

analysis_date = df["order_datetime"].max()
churn_threshold = analysis_date - pd.Timedelta(days=90)

churned_customers = last_purchase[last_purchase < churn_threshold]

churn_rate = (
    churned_customers.count() / last_purchase.count()
)


In [26]:
customer_ltv = (
    df.groupby("customer_id")["order_revenue"]
               .sum()
)

average_ltv = customer_ltv.mean()
